### GOES and AIA flare catalog production - Goal 1
This notebook contains some sample code to load up the AIA flare catalog, extract the entries which have an associate M/X GOES flare, and then plots the corresponding peak in each AIA channel along with markers for the start, peak and end times. You'll want to also have a copy of the catalog open so that you can make edits to it as needed. As you get accustomed to the data, it may be easier to transfer this code to a script, plot all of the peaks and then go through the generated plots one by one. 

In [1]:
## import packages
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
from datetime import datetime,timedelta
from helper import *

In [2]:
## load the AIA flare catalog 
flares = pd.read_csv('aia_flares_catalog_7_pred.csv')
lams = ['193','171','304','1600','131','94']

# convert all time columns to datetime objects
flares['aia_max_start_time'] = pd.to_datetime(flares['aia_max_start_time'])
flares['aia_min_end_time'] = pd.to_datetime(flares['aia_min_end_time'])
flares['goes_start_time'] = pd.to_datetime(flares['goes_start_time'])
flares['goes_end_time'] = pd.to_datetime(flares['goes_end_time'])
for lam in lams:
    flares[lam+'_start_time'] = pd.to_datetime(flares[lam+'_start_time'])
    flares[lam+'_peak_time'] = pd.to_datetime(flares[lam+'_peak_time'])
    flares[lam+'_end_time'] = pd.to_datetime(flares[lam+'_end_time'])

flares['mean_start_time']  = flares['aia_max_start_time']+flares[[lam+'_start_time'for lam in lams]].subtract(flares['aia_max_start_time'],axis=0).mean(axis=1)
flares['mean_peak_time']  = flares['aia_max_start_time']+flares[[lam+'_peak_time'for lam in lams]].subtract(flares['aia_max_start_time'],axis=0).mean(axis=1)
flares['mean_end_time']  = flares['aia_max_start_time']+flares[[lam+'_end_time'for lam in lams]].subtract(flares['aia_max_start_time'],axis=0).mean(axis=1)

# filter only events with associated GOES flare entry
flares_goes = flares[flares['goes_flare_ind'].notnull()]
# filter only M/X flares as given by GOES
flares_goes = flares_goes[np.logical_or(flares_goes['CMX']=='M',flares_goes['CMX']=='X')]
print(len(flares_goes),'flares in the AIA catalog with GOES M/X labels')


330 flares in the AIA catalog with GOES M/X labels


320


In [3]:
aia_dir = '/srv/data/sdo_sharps/aia_timeseries/' # might need to change this to the path to the aia timeseries data

In [76]:
## loop through the catalog and plot event
#i=np.arange(0,330) #there are 330 flares in the catalogue (index 0-329)
# for i in range(len(flares_goes)):
for i in range(len(flares_goes)):
    flare = flares_goes.iloc[i]
# load AIA timeseries data
    with open(aia_dir+'sharp_'+str(flare['SHARP'])+'_aia_sum_intensity', "rb") as fp:   #Pickling
        aia_data = pickle.load(fp)
    with open(aia_dir+'sharp_'+str(flare['SHARP'])+'_aia_times', "rb") as fp:   #Pickling
        aia_times = pickle.load(fp)

    plot_flare(flare,aia_times,aia_data,window=60);
    output_filename = f'timeseries_data/plot-{i}.png'
    plt.savefig(output_filename) #save figures to directory
    plt.close('all')
# break


In [13]:
i = 65
flare = flares_goes.iloc[i]
# load AIA timeseries data
with open(aia_dir+'sharp_'+str(flare['SHARP'])+'_aia_sum_intensity', "rb") as fp:   #Pickling
    aia_data = pickle.load(fp)
with open(aia_dir+'sharp_'+str(flare['SHARP'])+'_aia_times', "rb") as fp:   #Pickling
    aia_times = pickle.load(fp)

plot_flare(flare,aia_times,aia_data,window=5);
output_filename = f'timeseries_data/plot-{i}.png'
plt.savefig(output_filename, facecolor = 'white') #save figures to directory
plt.close('all')